### The goal of this notebook is to demonstrate the following:
- programmatic query of calibration files for any of the active instruments
- using the results, chunk all of the darks into sets of darks covering a ~30 day period
    - these chunks of darks will then be combined to reject cosmic rays
    - For the time being I will **ignore** anneal boundaries. All cosmic rays affecting only a few pixels are rejected and so any hot pixels that are cured will be ignored across the anneal boundary will be removed
#### Author: Nate Miles

In [ ]:
from astroquery.mast import Observations
from collections import defaultdict
from astropy.time import Time
import pandas as pd

- Originally, I was querying for every single dark every taken, then I was filtering that table by extension and date of observations.

- Instead, I should be filtering by date beforehand to get a much smaller table and then filter that table by extension to get only the RAW and SPT dataproducts

- In order to acheive this in a generalized manner without having to hard code dates, I will query for all darks, sort by date and then extract the first and last values to get the start and stop times

In [ ]:
#Observations.enable_s3_hst_dataset()

In [ ]:
obsTable = Observations.query_criteria(project='HST',
                                      instrument_name = 'WFPC2/WFC',
                                      obstype='cal',
                                      target_name = 'DARK')
df = obsTable.to_pandas()

In [ ]:
df.info()

In [ ]:
df.head()


In [ ]:
start = Time(min(obsTable['t_min']), format='mjd')
stop = Time(max(obsTable['t_min']), format='mjd')
print(start.iso, stop.iso)
# very roundabout way of generating a list of MJD dates separated by a month
dates = pd.date_range(start=start.iso, end=stop.iso, freq='1MS')

dates = [Time(date.date().isoformat(),format='iso') for date in dates]

In [ ]:
dates

In [ ]:
date_ranges = zip(dates[::2], dates[1::2])

In [ ]:
dates = next(date_ranges)

In [ ]:
obsTable = Observations.query_criteria(project='HST',
                                      instrument_name='WFPC2/WFC',
                                      obstype ='cal',
                                      target_name='DARK',
                                      t_min=[dates[0].mjd, dates[1].mjd],
                                      t_exptime=[800, 2000])

In [ ]:
products = Observations.get_product_list(obsTable)

In [ ]:
products

In [ ]:
products.columns

In [ ]:
filtered_products = Observations.filter_products(products,
                                                 mrp_only=False,
                                                 productSubGroupDescription=['D0M','D1M'])

In [ ]:
filtered_products


In [ ]:
filtered_products.columns

In [ ]:
download_list = filtered_products['obsID'].tolist()

In [ ]:
download_list

In [ ]:
manifest = Observations.download_products(download_list[:10],
                                          download_dir='./../crrejtab/WFPC2/',
                                          mrp_only=False,
                                         productSubGroupDescription=['D0M','D1M'])